In [13]:
import os
import numpy as np
import pandas as pd
import re
import time
import math
import jieba
from scipy.sparse import lil_matrix

In [2]:
file_id = list()  # id to path
with open('model/file-list') as f:
    for line, doc in enumerate(f):
        file_id.append(doc.split('/')[-1].strip().lower())
file_num = len(file_id)
print(len(file_id), file_id[0])

46972 cdn_loc_0001457


In [3]:
file_len = dict()  # path to len
for root, dirs, files in os.walk('CIRB010/'):
    for file in files:
        path = root + '/' + file
        with open(path) as f:
            content = f.read()
            content = re.sub(r'<\/*[a-z]*>', '', content)
            tmp = list(filter(None, content))
            #print(len(tmp))
            file_len[file.lower()] = len(content)
avdl = sum(file_len.values())/file_num
print('Average Doc Length: {}'.format(avdl))

Average Doc Length: 862.501064463936


In [4]:
vocab = dict()
with open('model/vocab.all') as vfile:
    vfile.readline()
    for line, word in enumerate(vfile):
        vocab[int(line)+1] = word.strip()

print(vocab[1])

Copper


In [5]:
start = time.time()
inverted = dict()  # word to count
vid = 0
with open('model/inverted-file') as f:
    for line in f:
        vid1, vid2, n = line.strip().split()
        vid1 = int(vid1)
        vid2 = int(vid2)
        if vid2 != -1:
            word = vocab[vid1]+vocab[vid2]
        else:
            word = vocab[vid1]
        Count = dict()  # DocId to count of word
        for i in range(int(n)):
            DocID, count = f.readline().strip().split()
            Count[int(DocID)] = int(count)
            #print(DocID, freq[int(DocID)])
        inverted[word] = Count
        vid += 1
print("Took {}".format(time.time()-start))

Took 34.85046935081482


In [67]:
df_ans = pd.read_csv('queries/ans_train.csv')

k = 1.2
b = 0.75
alpha = 0.8
beta = 0.1
gamma = 0.1
upgrade_num = 3

related = list()
query_id = list()

with open('queries/query-test.xml') as f:
    content = f.read()
    for qid, sub in enumerate(content.split('<topic>')[1:]):
        sub = re.sub(r'\n', '', sub)
        n, t, q, nar, concepts = list(filter(None, re.split(r'<\/*[a-z]*>', sub)))
        concepts = re.split(r'、|。', concepts)[:-1]
        
        query_id.append(n[-3:])
        
        for concept in concepts:
            if len(concept) > 2:
                for i in range(len(concept)-1):
                    concepts.append(concept[i:i+2])
    
        dv = np.zeros((file_num, len(concepts)))  # doc score
        qv = np.zeros(len(concepts))
        for i, word in enumerate(concepts):
            if len(word) > 2:
                continue
            try:
                df = sum(inverted[word].values())
                IDF = math.log((file_num-df+0.5)/(df+0.5))
                qv[i] += IDF
                for occur, count in inverted[word].items():
                    length = file_len[file_id[occur]]
                    TF = (k+1)*count/(count+k*(1-b+b*length/avdl))
                    #docw[occur][i] += TF/(1-b+b*length/avdl) * IDF
                    dv[occur][i] += TF * IDF
            except:
                continue
        #sim = np.sum(dv, 1)
        sim = np.dot(dv, qv)
        sim[np.isnan(sim)] = 0
        
        rid = [x for x in np.argsort(sim)[::-1][:100]]
        irid = [x for x in np.argsort(sim)[::-1][100:200]]
        topK = [file_id[x] for x in rid]
        bottomK = [file_id[x] for x in irid]
        
        '''hit = 0
        MAP = 0
        ans = df_ans.loc[qid]['retrieved_docs'].strip().split()
        for i, d in enumerate(topK):
            if d in ans:
                hit += 1
                MAP += hit/(i+1)
        print('MAP = {}'.format(MAP/(hit)))'''
        
        
        for i in range(upgrade_num):
            qv = alpha*qv + beta*np.sum([dv[x] for x in rid], 0)/100 - gamma*np.sum([dv[x] for x in irid], 0)/100
            sim = np.dot(dv, qv)
            sim[np.isnan(sim)] = 0

            rid = [x for x in np.argsort(sim)[::-1][:100]]
            irid = [x for x in np.argsort(sim)[::-1][100:200]]
            topK = [file_id[x] for x in rid]
            bottomK = [file_id[x] for x in irid]

            '''hit = 0
            MAP = 0
            ans = df_ans.loc[qid]['retrieved_docs'].strip().split()
            for j, d in enumerate(topK):
                if d in ans:
                    hit += 1
                    MAP += hit/(j+1)
            print('After upgrade {}times: MAP = {}'.format(i+1, MAP/(hit)))'''
        
        related.append(' '.join(topK))

In [68]:
df_ans = pd.DataFrame({'query_id': query_id, 'retrieved_docs': related})

df_ans.to_csv('test.csv', index=False)

!kaggle competitions submit -c wm-2019-vsm-model -f test.csv -m "doc length normalization, b=0.75"

100%|██████████████████████████████████████| 31.4k/31.4k [00:09<00:00, 3.56kB/s]
Successfully submitted to WM 2019 - VSM Model

In [ ]:
!